In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:0


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/1D FODE/atanh'

In [4]:
# !pip install smt

In [5]:
lr_tune = np.array([0.05,0.1,0.25,0.5,1])

In [6]:
def true_2D_4(xt): #True function for 2D_4 Heat Transfer in a rod x \in [0,1] t \in [0,0.1]
    term1 = 4*u0/np.pi
    
    resol_n = 10000
    
    x = xt[:,0].reshape(-1,1)
    t = xt[:,1].reshape(-1,1)

    u = np.zeros((np.shape(xt)[0],1))
    
    for i in range(resol_n):
        j = 2*i-1
        term2 = np.sin(j*np.pi*x)/j
        term3 = np.exp(-1*np.square(j*np.pi)*t)
        
        u = u + term2*term3
        
    u = term1*u
    
    return u

In [7]:
u0 = 50.0
loss_thresh = 0.1

x_ll = np.array(0.0)
x_ul = np.array(1.0)

x = np.linspace(x_ll,x_ul,100).reshape(-1,1)
t = np.linspace(0,0.1,100).reshape(-1,1)

X,T = np.meshgrid(x,t)

X = X.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xt = np.hstack((X,T))

u_true = true_2D_4(xt)
u_true_norm = np.linalg.norm(u_true,2)


lb_xt = xt[0]
ub_xt = xt[-1]

xt_test_tensor = torch.from_numpy(xt).float().to(device)

In [8]:
def trainingdata(N_f,N_train,seed):
    
    np.random.seed(seed)
    
    #X_Train
    np.random.seed(seed)
    x_train = np.random.uniform(x_ll,x_ul,(N_train,1))
    t_train = np.random.uniform(0,0.1,(N_train,1))
    
    xt_train = np.hstack((x_train,t_train))
    u_train = true_2D_4(xt_train)
    

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)

    samples = sampling(N_f)
    
    xt_coll = lb_xt + (ub_xt - lb_xt)*samples
    
    xt_coll = np.vstack((xt_coll)) # append training points to collocation points 

    return xt_coll, xt_train, u_train

In [9]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Sigmoid()

     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self.iter = 0
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
        
        
        self.beta = Parameter(torch.ones((50,len(layers)-2)))
        self.beta.requiresGrad = True
    
        self.lambda1 = Parameter(torch.tensor(0.0))
        self.lambda1.requiresGrad = True
        

            
    'foward pass'
    def forward(self,xt):
        if torch.is_tensor(xt) != True:         
            xt = torch.from_numpy(xt)                
        
        ubxt = torch.from_numpy(ub_xt).float().to(device)
        lbxt = torch.from_numpy(lb_xt).float().to(device)
    
                      
        #preprocessing input 
        xt = (xt - lbxt)/(ubxt - lbxt)
        
        #convert to float
        a = xt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = z*self.activation(self.beta[:,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    
    def loss_PDE(self, xt_coll,f_hat):
        
        g = xt_coll.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_t = autograd.grad(u,g,torch.ones([xt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_xx_tt = autograd.grad(u_x_t,g,torch.ones(xt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        du_dx = u_x_t[:,[0]]
        
        d2u_dx2 = u_xx_tt[:,[0]]
                
        du_dt = u_x_t[:,[1]]
        
        f = du_dt - self.lambda1*d2u_dx2
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xt_coll,f_hat, xt_train, u_train):

        loss_f = self.loss_PDE(xt_coll,f_hat)
        
        loss_train = self.loss_function(self.forward(xt_train),u_train)
        
        loss_val = loss_f + loss_train
        
        #print(self.iter,"train_loss",loss_train.cpu().detach().numpy(),"F Loss",(loss_f).cpu().detach().numpy())
        return loss_val
     
    'callable for optimizer'                                    
    
    def test(self):
        u_pred = self.forward(xt_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

    def test_loss(self):
        u_pred = self.test()
               
        test_mse = np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))
        test_re = np.linalg.norm(u_pred.reshape(-1,1) - u_true.reshape(-1,1),2)/u_true_norm
        
        return test_mse, test_re 

In [10]:
def train_step(xt_coll,f_hat, xt_train, u_train,seed):    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xt_coll,f_hat, xt_train, u_train)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [11]:
def data_update(loss_np):
    train_loss.append(loss_np)
    beta_val.append(PINN.beta.cpu().detach().numpy())
    lambda1_val.append(PINN.lambda1.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [12]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0
    
    xt_coll, xt_train, u_train = trainingdata(N_f,N_train,123)
    
    xt_coll = torch.from_numpy(xt_coll).float().to(device)
    xt_train = torch.from_numpy(xt_train).float().to(device)
    u_train = torch.from_numpy(u_train).float().to(device)
    
    
    f_hat = torch.zeros(xt_coll.shape[0],1).to(device)
    nan_flag = 0
    
    for i in range(max_iter):
        train_step(xt_coll,f_hat, xt_train, u_train,i)

        loss_np = PINN.loss(xt_coll,f_hat, xt_train, u_train).cpu().detach().numpy()
        
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1],"Lambda1",lambda1_val[-1])

        if(np.isnan(loss_np)):
            nan_flag =1
            print("NAN BREAK!")
            break
    
    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))

In [13]:
nan_tune = []
for tune_reps in range(5):
    label = "inv_HT_swish_tune" + str(tune_reps)
    max_reps = 10 #10
    max_iter = 75 #75

    train_loss_full = []
    test_mse_full = []
    test_re_full = []
    beta_full = []
   
    lambda1_full = []
    elapsed_time= np.zeros((max_reps,1))
    
    time_threshold = np.empty((max_reps,1))
    time_threshold[:] = np.nan
    epoch_threshold = max_iter*np.ones((max_reps,1))


    for reps in range(max_reps):
        print(label)
        'Generate Training data'
        print(reps)
        torch.manual_seed(reps*36)
        
        train_loss = []
        test_mse_loss = []
        test_re_loss = []   
        beta_val = []
        
        lambda1_val = []

        N_f = 50000 #Total number of collocation points 
        N_train = 500

        layers = np.array([2,50,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
        
        PINN = Sequentialmodel(layers)

        PINN.to(device)

        'Neural Network Summary'
        print(PINN)

        params = list(PINN.parameters())

        optimizer = torch.optim.LBFGS(PINN.parameters(), lr=lr_tune[tune_reps], 
                                  max_iter = 10, 
                                  max_eval = 15, 
                                  tolerance_grad = -1, 
                                  tolerance_change = -1, 
                                  history_size = 100, 
                                  line_search_fn = 'strong_wolfe')
        
        
        nan_flag = train_model(max_iter,reps)

        torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
        train_loss_full.append(train_loss)
        test_mse_full.append(test_mse_loss)
        test_re_full.append(test_re_loss)
        #elapsed_time[reps] = time.time() - start_time
        beta_full.append(beta_val)

        lambda1_full.append(lambda1_val)
        
        if(nan_flag == 1):
            nan_tune.append(tune_reps)
            break

        print('Training time: %.2f' % (elapsed_time[reps]))

    mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "beta": beta_full,"lambda1": lambda1_full, "label": label}
    savemat(label+'.mat', mdic) 


inv_HT_swish_tune0
0
Sequentialmodel(
  (activation): Sigmoid()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=50, bias=True)
    (10): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 838.04877 Test MSE 858.267842986848 Test RE 0.49317447521322566 Lambda1 -2.7418824e-07
1 Train Loss 838.04877 Test MSE 858.2678212496938 Test RE 0.4931744689679668 Lambda1 -2.9793296e-07
2 Train Loss 83

74 Train Loss 838.0486 Test MSE 858.2676516458465 Test RE 0.49317442023941765 Lambda1 -4.3530383e-07
Training time: 155.02
Training time: 155.02
inv_HT_swish_tune0
1
Sequentialmodel(
  (activation): Sigmoid()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=50, bias=True)
    (10): Linear(in_features=50, out_features=1, bias=True)
  )
)
1
0 Train Loss 838.79974 Test MSE 859.829898873315 Test RE 0.49362306238

0 Train Loss 840.8417 Test MSE 862.6630928316092 Test RE 0.4944356531756279 Lambda1 2.4666165e-06
1 Train Loss 838.0969 Test MSE 858.4571479951354 Test RE 0.4932288610622396 Lambda1 2.1281141e-06
2 Train Loss 838.07605 Test MSE 858.3259134612 Test RE 0.4931911590442331 Lambda1 1.9005313e-06
3 Train Loss 838.07477 Test MSE 858.3042909094149 Test RE 0.4931849468826457 Lambda1 1.7312166e-06
4 Train Loss 838.07465 Test MSE 858.3003251680808 Test RE 0.49318380751621754 Lambda1 1.6500305e-06
5 Train Loss 838.07434 Test MSE 858.2968086115511 Test RE 0.4931827971993948 Lambda1 1.5474662e-06
6 Train Loss 838.074 Test MSE 858.2911278467852 Test RE 0.49318116509493787 Lambda1 1.335076e-06
7 Train Loss 838.07196 Test MSE 858.2617307416783 Test RE 0.49317271911302024 Lambda1 -3.8571903e-07
8 Train Loss 838.06757 Test MSE 858.2210150392402 Test RE 0.4931610209846321 Lambda1 -5.1892976e-06
9 Train Loss 838.0379 Test MSE 858.1345232389923 Test RE 0.49313616989236997 Lambda1 -2.8673268e-05
10 Train Los

0 Train Loss 838.51776 Test MSE 859.3875448935053 Test RE 0.49349606970055565 Lambda1 2.6951108e-07
1 Train Loss 838.05945 Test MSE 858.3171784032896 Test RE 0.49318864947052216 Lambda1 2.9899172e-07
2 Train Loss 838.05664 Test MSE 858.2788747196801 Test RE 0.49317764470839687 Lambda1 3.3015294e-07
3 Train Loss 838.0545 Test MSE 858.2502350588311 Test RE 0.49316941628828215 Lambda1 4.3680402e-07
4 Train Loss 838.0396 Test MSE 858.1427457133328 Test RE 0.4931385324530645 Lambda1 2.0646523e-06
5 Train Loss 837.931 Test MSE 857.7833667675741 Test RE 0.4930352616595927 Lambda1 0.0004920985
6 Train Loss 837.885 Test MSE 857.8711809616748 Test RE 0.4930604978601436 Lambda1 0.0017639116
7 Train Loss 837.8846 Test MSE 857.8807876761582 Test RE 0.4930632585765953 Lambda1 0.0018556062
8 Train Loss 837.88446 Test MSE 857.8842022372168 Test RE 0.4930642398276916 Lambda1 0.0019063149
9 Train Loss 837.88434 Test MSE 857.887645972114 Test RE 0.49306522946057024 Lambda1 0.0019751962
10 Train Loss 837.

0 Train Loss 838.13104 Test MSE 858.1013830488736 Test RE 0.4931266476183158 Lambda1 -1.16651755e-07
1 Train Loss 838.0646 Test MSE 858.2685968862116 Test RE 0.4931746918145142 Lambda1 -1.5335483e-07
2 Train Loss 838.06433 Test MSE 858.2738676772005 Test RE 0.4931762061520654 Lambda1 -1.5412228e-07
3 Train Loss 838.06415 Test MSE 858.2779320333661 Test RE 0.4931773738687214 Lambda1 -1.5125782e-07
4 Train Loss 838.0634 Test MSE 858.288359492266 Test RE 0.49318036973462026 Lambda1 -1.1993005e-07
5 Train Loss 838.0617 Test MSE 858.3115592511402 Test RE 0.49318703508678086 Lambda1 1.04903734e-07
6 Train Loss 838.0541 Test MSE 858.3579775228216 Test RE 0.4932003709087378 Lambda1 2.7488825e-06
7 Train Loss 837.978 Test MSE 858.369191012492 Test RE 0.4932035924546121 Lambda1 0.00011305725
8 Train Loss 837.86145 Test MSE 857.8662728112783 Test RE 0.4930590873811619 Lambda1 0.0043454175
9 Train Loss 837.85925 Test MSE 857.8704009763643 Test RE 0.49306027371225203 Lambda1 0.005794847
10 Train Lo

...

7 Train Loss 718.9536 Test MSE 717.7706195999767 Test RE 0.45100567782527934 Lambda1 -0.068573624
8 Train Loss 548.0785 Test MSE 500.65232537976607 Test RE 0.3766667540088279 Lambda1 -0.014808941
9 Train Loss 377.43054 Test MSE 365.7031917990074 Test RE 0.32192414137721775 Lambda1 -0.009733399
10 Train Loss 300.0711 Test MSE 312.17875266976284 Test RE 0.2974341571465618 Lambda1 -0.003158619
11 Train Loss 271.2805 Test MSE 289.8964898534521 Test RE 0.28662274374633856 Lambda1 -0.0038145392
12 Train Loss 265.36362 Test MSE 284.02149970817953 Test RE 0.28370355561166166 Lambda1 -0.0026326035
13 Train Loss 261.4023 Test MSE 282.5159263849319 Test RE 0.2829506124996882 Lambda1 -0.0022991416
14 Train Loss 257.90057 Test MSE 282.2641483050743 Test RE 0.28282450165996886 Lambda1 -0.00047587117
15 Train Loss 257.30484 Test MSE 281.68984197687513 Test RE 0.28253663193432077 Lambda1 -6.480379e-05
16 Train Loss 257.16245 Test MSE 281.83170122967994 Test RE 0.2826077658281621 Lambda1 -7.6002914e-05

8 Train Loss 837.6075 Test MSE 857.771431787369 Test RE 0.49303183166387066 Lambda1 -0.15938672
9 Train Loss 837.6075 Test MSE 857.771431787369 Test RE 0.49303183166387066 Lambda1 -0.15938672
10 Train Loss 837.6075 Test MSE 857.771431787369 Test RE 0.49303183166387066 Lambda1 -0.15938672
11 Train Loss 837.6075 Test MSE 857.771431787369 Test RE 0.49303183166387066 Lambda1 -0.15938672
12 Train Loss 837.6075 Test MSE 857.771431787369 Test RE 0.49303183166387066 Lambda1 -0.15938672
13 Train Loss 837.6075 Test MSE 857.771431787369 Test RE 0.49303183166387066 Lambda1 -0.15938672
14 Train Loss 837.6075 Test MSE 857.771431787369 Test RE 0.49303183166387066 Lambda1 -0.15938672
15 Train Loss 837.6075 Test MSE 857.771431787369 Test RE 0.49303183166387066 Lambda1 -0.15938672
16 Train Loss 837.6075 Test MSE 857.771431787369 Test RE 0.49303183166387066 Lambda1 -0.15938672
17 Train Loss 837.6075 Test MSE 857.771431787369 Test RE 0.49303183166387066 Lambda1 -0.15938672
18 Train Loss 837.6075 Test MSE 

9 Train Loss 286.56744 Test MSE 297.8795004055155 Test RE 0.29054237281506307 Lambda1 -0.0009810154
10 Train Loss 268.00735 Test MSE 286.8252386833322 Test RE 0.28510041695620814 Lambda1 -0.0051085437
11 Train Loss 265.2935 Test MSE 284.2813469325387 Test RE 0.28383330411309937 Lambda1 -0.0056731105
12 Train Loss 261.32224 Test MSE 281.22885190507316 Test RE 0.2823053493423655 Lambda1 -0.002261241
13 Train Loss 260.60043 Test MSE 279.34072667846834 Test RE 0.28135607701512005 Lambda1 0.0014019766
14 Train Loss 258.12946 Test MSE 274.57929066565964 Test RE 0.2789478767126144 Lambda1 0.01048736
15 Train Loss 256.8397 Test MSE 274.2553752316654 Test RE 0.27878329367780785 Lambda1 0.011551838
16 Train Loss 256.41827 Test MSE 272.0003608398823 Test RE 0.27763480581305916 Lambda1 0.015955104
17 Train Loss 253.91618 Test MSE 268.98772878991235 Test RE 0.27609300561740663 Lambda1 0.028071666
18 Train Loss 248.59232 Test MSE 258.3849466282423 Test RE 0.2705968737263652 Lambda1 0.066970915
19 Tr

11 Train Loss 837.6879 Test MSE 858.2124534807334 Test RE 0.493158561106937 Lambda1 -0.06898803
12 Train Loss 837.6879 Test MSE 858.2124534807334 Test RE 0.493158561106937 Lambda1 -0.06898803
13 Train Loss 837.6879 Test MSE 858.2124534807334 Test RE 0.493158561106937 Lambda1 -0.06898803
14 Train Loss 837.6879 Test MSE 858.2124534807334 Test RE 0.493158561106937 Lambda1 -0.06898803
15 Train Loss 837.6879 Test MSE 858.2124534807334 Test RE 0.493158561106937 Lambda1 -0.06898803
16 Train Loss 837.6879 Test MSE 858.2124534807334 Test RE 0.493158561106937 Lambda1 -0.06898803
17 Train Loss 837.6879 Test MSE 858.2124534807334 Test RE 0.493158561106937 Lambda1 -0.06898803
18 Train Loss 837.6879 Test MSE 858.2124534807334 Test RE 0.493158561106937 Lambda1 -0.06898803
19 Train Loss 837.6879 Test MSE 858.2124534807334 Test RE 0.493158561106937 Lambda1 -0.06898803
20 Train Loss 837.6879 Test MSE 858.2124534807334 Test RE 0.493158561106937 Lambda1 -0.06898803
21 Train Loss 837.6879 Test MSE 858.2124

13 Train Loss 359.33453 Test MSE 302.11068279339196 Test RE 0.2925985784575564 Lambda1 0.21999246
14 Train Loss 298.96936 Test MSE 261.1931187209876 Test RE 0.2720633467332648 Lambda1 0.2666962
15 Train Loss 248.43343 Test MSE 225.50302164135198 Test RE 0.2527931801829288 Lambda1 0.32650438
16 Train Loss 211.48422 Test MSE 174.2987439768697 Test RE 0.22224720062200753 Lambda1 0.3701516
17 Train Loss 176.08882 Test MSE 151.7814522656944 Test RE 0.20739511491056495 Lambda1 0.43772084
18 Train Loss 140.61763 Test MSE 126.85083910186889 Test RE 0.18959891286246044 Lambda1 0.47019833
19 Train Loss 122.88477 Test MSE 120.09150350595485 Test RE 0.1844783101114369 Lambda1 0.49125287
20 Train Loss 111.402176 Test MSE 99.98187643710477 Test RE 0.16832546087844136 Lambda1 0.5577085
21 Train Loss 80.052765 Test MSE 72.46947018471893 Test RE 0.14330677205800665 Lambda1 0.6099371
22 Train Loss 69.04623 Test MSE 65.32316656214147 Test RE 0.1360575935110873 Lambda1 0.64492553
23 Train Loss 62.806995 T

In [14]:
import scipy.io as sio

In [22]:
for tune_reps in range(5):
    label = "inv_HT_swish_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["lambda1"])
    print(np.mean(re[:,-1]))

0.3803765
0.5332857
0.6176917
[[1.0936446e-05 5.1908372e-03 2.6749015e-01 8.3244300e-01 7.1130824e-01
  4.3814161e-01 5.0620049e-01 5.8139008e-01 7.2378534e-01 8.4669340e-01
  9.0908474e-01 9.4020641e-01 9.3450940e-01 9.6880680e-01 1.0148460e+00
  1.0518965e+00 1.0857978e+00 1.1224855e+00 1.1421726e+00 1.1773984e+00
  1.1935776e+00 1.2131006e+00 1.2148787e+00 1.2041231e+00 1.1529937e+00
  1.1451490e+00 1.1265110e+00 1.1240835e+00 1.1331701e+00 1.1316785e+00
  1.0960108e+00 1.0456393e+00 1.0022391e+00 9.7977114e-01 9.9185133e-01
  1.0047909e+00 9.8794007e-01 9.6512049e-01 9.5608795e-01 9.4431275e-01
  9.3384027e-01 9.2426163e-01 9.3659365e-01 9.6629703e-01 9.7704625e-01
  9.8526388e-01 1.0508255e+00 1.0847062e+00 1.1020237e+00 1.0776469e+00
  1.0739287e+00 1.0513834e+00 1.0092171e+00 9.9098402e-01 9.6912360e-01
  9.6092439e-01 9.6978724e-01 9.9861151e-01 1.0082142e+00 1.0256081e+00
  1.0191369e+00 1.0255429e+00 1.0238643e+00 1.0226312e+00 1.0216644e+00
  1.0228508e+00 1.0223229e+00 1.02